In [2]:
!python -V

/bin/sh: 1: python: not found


In [1]:
import argparse
import os
import shutil
import warnings
from pprint import pprint
import mmcv
from depth.datasets.pipelines import Compose
import torch
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import get_dist_info, init_dist, load_checkpoint, wrap_fp16_model
from mmcv.utils import DictAction

from depth.apis import multi_gpu_test, single_gpu_test
from depth.datasets import build_dataloader, build_dataset
from depth.models import build_depther

import numpy as np
import pickle
from pathlib import Path
import matplotlib.pyplot as plt

In [ ]:
path_to_images = Path("../dataset/block-insertion-test/color/000000-1.pkl")
path_to_config = Path("configs/binsformer/binsformer_swint_w7_nyu.py")
path_to_checkpoint = Path("../checkpoints/binsformer_swint_nyu_converted.pth")

batch = torch.Tensor(pickle.load(open(path_to_images, "rb"))[0])
image = batch[0]
img_metas = [
    {
        "pad_shape": tuple(_.shape),
        "img_shape": tuple(_.shape),
        "ori_shape": tuple(_.shape),
        "scale_factor": 1,
        "img_norm_cfg": {
            "mean": _.mean(axis=(0, 1)),
            "std": _.mean(axis=(0, 1)),
        },
    }
    for _ in batch
]
# plt.imshow(image)
outputs = model.forward([batch.permute((0, 3, 1, 2))], [img_metas], return_loss=False)
# ori_shapes = [_['ori_shape'] for _ in img_metas]
# print(ori_shapes, [shape == ori_shapes[0] for shape in ori_shapes])
# assert all(shape == ori_shapes[0] for shape in ori_shapes)

In [121]:
torch.transpose(batch, (0, 1, 2, 3), (0, 3, 1, 2))

TypeError: transpose() received an invalid combination of arguments - got (Tensor, tuple, tuple), but expected one of:
 * (Tensor input, name dim0, name dim1)
      didn't match because some of the arguments have invalid types: (Tensor, !tuple!, !tuple!)
 * (Tensor input, int dim0, int dim1)
      didn't match because some of the arguments have invalid types: (Tensor, !tuple!, !tuple!)


In [57]:
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True
)

eval_pipeline = [
    dict(type="LoadImageFromFile"),
    dict(type="RandomFlip", prob=0.0),  # set to zero
    dict(type="Normalize", **img_norm_cfg),
    dict(type="ImageToTensor", keys=["img"]),
    dict(
        type="Collect",
        keys=["img"],
        meta_keys=(
            "filename",
            "ori_filename",
            "ori_shape",
            "img_shape",
            "pad_shape",
            "scale_factor",
            "flip",
            "flip_direction",
            "img_norm_cfg",
            "cam_intrinsic",
        ),
    ),
]
Compose(eval_pipeline)

Compose(
    LoadImageFromFile(to_float32=False,color_type='color',imdecode_backend='cv2')
    RandomFlip(prob=0.0)
    Normalize(mean=[123.675 116.28  103.53 ], std=[58.395 57.12  57.375], to_rgb=True)
    ImageToTensor(keys=['img'])
    Collect(keys=['img'], meta_keys=('filename', 'ori_filename', 'ori_shape', 'img_shape', 'pad_shape', 'scale_factor', 'flip', 'flip_direction', 'img_norm_cfg', 'cam_intrinsic'))
)

In [3]:
cfg = mmcv.Config.fromfile(path_to_config)
print(cfg.model.pretrained, "here", cfg.data.test)
# cfg.model.pretrained = None
# cfg.data.test.test_mode = True
# pprint(cfg._cfg_dict)
model = build_depther(cfg.model, test_cfg=cfg.get("test_cfg"))
checkpoint = load_checkpoint(model, str(path_to_checkpoint), map_location="cpu")

https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth here {'type': 'NYUBinFormerDataset', 'data_root': 'data/nyu/', 'depth_scale': 1000, 'split': 'nyu_test.txt', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (480, 640), 'flip': True, 'flip_direction': 'horizontal', 'transforms': [{'type': 'RandomFlip', 'direction': 'horizontal'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}], 'garg_crop': False, 'eigen_crop': True, 'min_depth': 0.001, 'max_depth': 10}


/home/matias/RECVIS/RECVIS-transporter-networks/Monocular-Depth-Estimation-Toolbox/depth/models/backbones/swin.py:612: UserWarning: DeprecationWarning: pretrained is a deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is a deprecated, '
/home/matias/.pyenv/versions/3.7.12/envs/RECVIS/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:342: UserWarning: The arguments `feedforward_channels` in BaseTransformerLayer has been deprecated, now you should set `feedforward_channels` and other FFN related arguments to a dict named `ffn_cfgs`. 
  f'The arguments `{ori_name}` in BaseTransformerLayer '
/home/matias/.pyenv/versions/3.7.12/envs/RECVIS/lib/python3.7/site-packages/mmcv/cnn/bricks/transformer.py:342: UserWarning: The arguments `ffn_dropout` in BaseTransformerLayer has been deprecated, now you should set `ffn_drop` and other FFN related arguments to a dict named `ffn_cfgs`. 
  f'The arguments `{ori_name}` in BaseTransformerLayer '
/home/mati

load checkpoint from local path: ../checkpoints/binsformer_swint_nyu_converted.pth


In [63]:
model.forward([batch], [img_metas], return_loss=False)

KeyError: 'pad_shape'